In [32]:
import advent
import numpy as np

maze = advent.get_lines(22)[:-2]
directions = advent.get_lines(22)[-1]

#maze = advent.get_lines('22small')[:-2]
#directions = advent.get_lines('22small')[-1]

max_len = max(len(line) for line in maze)
maze = np.array([[c for c in line.ljust(max_len)] for line in maze])

# Offset in the numpy array to move 1 step
compass = {'N': (-1, 0), 'S': (1, 0), 'W': (0, -1), 'E': (0, 1)}
print(max_len, maze.shape)


150 (200, 150)


In [33]:
tadd = lambda a, b: (a[0] + b[0], a[1] + b[1])

def step(maze, position, direction):
    y, x = tadd(position, compass[direction])
    y, x = y % maze.shape[0], x % maze.shape[1]
    if maze[(y, x)] == '.': return (y, x)
    if maze[(y, x)] == '#': return False
    # We hit a whitespace, just keep going
    return step(maze, (y, x), direction)

def full_step(maze, position, direction, n=1):
    while n > 0:
        new_position = step(maze, position, direction)
        if not new_position: return position # we hit a wall
        position, n = new_position, n-1
    return position
    
step(maze, (25, 40), 'W')

False

In [34]:
import re

steps = re.split('(?=L|R)', directions)

position, direction = (0, 50), 'E'
#position, direction = (0, 8), 'E' # for test example

lturn = {'N': 'W', 'W': 'S', 'S': 'E', 'E': 'N'}
rturn = {'N': 'E', 'E': 'S', 'S': 'W', 'W': 'N'}

# First step doesn't have a turn
#print(position, direction)
position = full_step(maze, position, direction, n=int(steps[0]))

for stepl in steps[1:]:
    turn = stepl[0]
    if turn == 'L': direction = lturn[direction]
    elif turn == 'R': direction = rturn[direction]
    else: raise ValueError(f"{stepl} doesnt start with L or R")
    #print(position, direction)
    position = full_step(maze, position, direction, n=int(stepl[1:]))


points = {'E': 0, 'S': 1, 'W': 2, 'N': 3}
print(position, direction)
print(1000 * (position[0]+1) + 4 * (position[1]+1) + points[direction])

(75, 82) E
76332


In [35]:
MAXL = 49

def getside(maze, side):
    if side == 1: return maze[150:200, 0:50]
    if side == 2: return maze[100:150, 0:50]
    if side == 3: return maze[100:150, 50:100]
    if side == 4: return maze[50:100, 50:100]
    if side == 5: return maze[0:50, 50:100]
    if side == 6: return maze[0:50, 100:150]
    raise ValueError(f"{side} is not a side!")

def left(side, y, x):
    # Returns side, (y, x), direction when you step one step left off side=side
    if x > 0: return side, (y, x-1), 'W'
    if side == 1: return 5, (0, y), 'S'
    if side == 2: return 5, (MAXL-y, 0), 'E'
    if side == 3: return 2, (y, MAXL), 'W'
    if side == 4: return 2, (0, y), 'S'
    if side == 5: return 2, (MAXL-y, 0), 'E'
    if side == 6: return 5, (y, MAXL), 'W'

def right(side, y, x):
    if x < MAXL: return side, (y, x+1), 'E'
    if side == 1: return 3, (MAXL, y), 'N'
    if side == 2: return 3, (y, 0), 'E'
    if side == 3: return 6, (MAXL-y, MAXL), 'W'
    if side == 4: return 6, (MAXL, y), 'N'
    if side == 5: return 6, (y, 0), 'E'
    if side == 6: return 3, (MAXL-y, MAXL), 'W'

def up(side, y, x):
    if y > 0: return side, (y-1, x), 'N'
    if side == 1: return 2, (MAXL, x), 'N'
    if side == 2: return 4, (x, 0), 'E'
    if side == 3: return 4, (MAXL, x), 'N'
    if side == 4: return 5, (MAXL, x), 'N'
    if side == 5: return 1, (x, 0), 'E'
    if side == 6: return 1, (MAXL, x), 'N'

def down(side, y, x):
    if y < MAXL: return side, (y+1, x), 'S'
    if side == 1: return 6, (0, x), 'S'
    if side == 2: return 1, (0, x), 'S'
    if side == 3: return 1, (x, MAXL), 'W'
    if side == 4: return 3, (0, x), 'S'
    if side == 5: return 4, (0, x), 'S'
    if side == 6: return 4, (x, MAXL), 'W'


In [38]:

# For part 2, position is a tuple (side, (y, x))
fn_dict = {'N': up, 'E': right, 'W': left, 'S': down}

def step_2(maze, position, direction):
    side, position = position
    y, x = position
    new_side, new_pos, new_dir = fn_dict[direction](side, y, x)
    #print(side, position, direction, new_side, new_pos, new_dir)

    maze_side = getside(maze, new_side)

    if maze_side[new_pos] == '.': return (new_side, new_pos), new_dir
    if maze_side[new_pos] == '#': return False, False

def full_step_2(maze, position, direction, n=1):
    while n > 0:
        new_position, new_direction = step_2(maze, position, direction)
        if not new_position: return position, direction # we hit a wall
        position, n, direction = new_position, n-1, new_direction
    return position, direction

In [42]:
import re

steps = re.split('(?=L|R)', directions)

position, direction = (5, (0, 0)), 'E'

lturn = {'N': 'W', 'W': 'S', 'S': 'E', 'E': 'N'}
rturn = {'N': 'E', 'E': 'S', 'S': 'W', 'W': 'N'}

# First step doesn't have a turn
#print(position, direction)
position, direction = full_step_2(maze, position, direction, n=int(steps[0]))

for stepl in steps[1:]:
    turn = stepl[0]
    if turn == 'L': direction = lturn[direction]
    elif turn == 'R': direction = rturn[direction]
    else: raise ValueError(f"{stepl} doesnt start with L or R")
    #print(position, direction)
    position, direction = full_step_2(maze, position, direction, n=int(stepl[1:]))


points = {'E': 0, 'S': 1, 'W': 2, 'N': 3}
print(position, direction)
# Output is 2, (43, 2)
# I didn't feel like writing a function so manually:
position = (143, 2)
print(1000 * (position[0]+1) + 4 * (position[1]+1) + points[direction])

(2, (43, 2)) E
144012
